In [ ]:
import os
import logging
import pandas as pd
import datasets
from ml_wrappers.model import OpenaiWrapperModel
from transformers import pipeline
import numpy as np

In [ ]:
def replace_error_chars(message:str):
    message = message.replace('`', '')
    return message

In [ ]:
dataset = datasets.load_dataset("squad", split="train")
dataset

In [ ]:
questions = []
context = []
answers = []
prompts = []
template = 'Answer the question given the context.\n\ncontext:\n{context}\n\nquestion:\n{question}'
for row in dataset:
    context.append(row['context'])
    questions.append(row['question'])
    answers.append(replace_error_chars(row['answers']['text'][0]))
    templated_prompt = template.format(context=row['context'], question=row['question'])
    prompts.append(templated_prompt)

In [ ]:
data = pd.DataFrame({
    'context': context,
    'questions': questions,
    # 'answers': answers,
    'prompt' : prompts})
test_data = data[:3]
data.head()

In [ ]:
class template(object):
    def __init__(self, model):
        self.model = model

    def predict(self, dataset):
        dummy = 'This is a dummy answer'
        return np.array([dummy for _ in range(len(dataset))])
        # template = 'Answer the question given the context.'
        # for i, (context, question) in enumerate(zip(dataset['context'], dataset['questions'])):
        #     templated_question = template + '\n\ncontext: ' + context + '\nquestion: ' + question
        #     if isinstance(dataset, pd.DataFrame):
        #         dataset.iloc[i]['questions'] = templated_question
        #     else:
        #         dataset['questions'] = templated_question
        # return self.model.predict(dataset)

# Model and Evaluator

In [ ]:
openai_model = OpenaiWrapperModel(
    os.getenv('OPENAI_API_TYPE'),
    os.getenv('OPENAI_API_ENDPOINT'),
    os.getenv('OPENAI_API_VERSION'),
    os.getenv('OPENAI_API_KEY'),
    engine='gpt-4')

pipeline_model = template(openai_model)

In [ ]:
eval_model = OpenaiWrapperModel(
    os.getenv('OPENAI_API_TYPE'),
    os.getenv('OPENAI_API_ENDPOINT'),
    os.getenv('OPENAI_API_VERSION'),
    os.getenv('OPENAI_API_KEY'),
    engine='gpt-4')

# RAI Insights

In [ ]:
from responsibleai_text import RAITextInsights, ModelTask
from raiwidgets import ResponsibleAIDashboard

In [ ]:
test_data.head()

In [ ]:
rai_insights = RAITextInsights(
    pipeline_model, test_data, None,
    task_type=ModelTask.GENERATIVE_TEXT)

In [ ]:
# TODO: Remove this once the insights object is updated to handle this
rai_insights.temp_questions = test_data['questions']
rai_insights.temp_context = test_data['context']
rai_insights.temp_eval_model = eval_model

In [ ]:
rai_insights.error_analysis.add()
# rai_insights.compute()

In [ ]:
ResponsibleAIDashboard(rai_insights)